# TLC Trip Record Data https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [1]:
import pandas as pd
import os

dir_ = os.getcwd()
data_dir_ = os.path.join(dir_, 'data')
file_jan = os.path.join(data_dir_, 'yellow_tripdata_2022-01.parquet')
file_feb = os.path.join(data_dir_, 'yellow_tripdata_2022-02.parquet')

In [76]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math

In [2]:
print("Current directory : ", dir_)
print("Data directory : ", data_dir_, '\n')
print("January File : ", file_jan)
print("February File : ", file_feb)

Current directory :  /home/dell/Documents/notebooks_mlops
Data directory :  /home/dell/Documents/notebooks_mlops/data 

January File :  /home/dell/Documents/notebooks_mlops/data/yellow_tripdata_2022-01.parquet
February File :  /home/dell/Documents/notebooks_mlops/data/yellow_tripdata_2022-02.parquet


In [6]:
!pip install pyarrow

     |████████████████████████████████| 39.0 MB 2.7 MB/s eta 0:00:01


In [7]:
jan_df = pd.read_parquet(file_jan)
feb_df = pd.read_parquet(file_feb)

jan_df.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,2.463931e+06,2.392428e+06,2.463931e+06,2.392428e+06,2.463931e+06,2.463931e+06,2.463931e+06,2.463931e+06,2.463931e+06,2.463931e+06,2.463931e+06,2.463931e+06,2.463931e+06,2.463931e+06,2.392428e+06,2.392428e+06
mean,1.707819e+00,1.389453e+00,5.372751e+00,1.415507e+00,1.660768e+02,1.635807e+02,1.194449e+00,1.294648e+01,1.006710e+00,4.914539e-01,2.385696e+00,3.749773e-01,2.967234e-01,1.916937e+01,2.282322e+00,8.249935e-02
std,5.021375e-01,9.829686e-01,5.478714e+02,5.917573e+00,6.546806e+01,7.079016e+01,5.001778e-01,2.558149e+02,1.236226e+00,8.291560e-02,2.830698e+00,1.680538e+00,4.374741e-02,2.559641e+02,7.432040e-01,3.125554e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-4.800000e+02,-4.500000e+00,-5.000000e-01,-1.252200e+02,-3.140000e+01,-3.000000e-01,-4.803000e+02,-2.500000e+00,-1.250000e+00
25%,1.000000e+00,1.000000e+00,1.040000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,7.200000e-01,0.000000e+00,3.000000e-01,1.130000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,1.740000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,2.000000e+00,0.000000e+00,3.000000e-01,1.469000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,1.000000e+00,3.130000e+00,1.000000e+00,2.340000e+02,2.360000e+02,1.000000e+00,1.400000e+01,2.500000e+00,5.000000e-01,3.000000e+00,0.000000e+00,3.000000e-01,2.002000e+01,2.500000e+00,0.000000e+00
max,6.000000e+00,9.000000e+00,3.061593e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,4.010923e+05,3.350000e+01,1.659000e+01,8.888800e+02,1.933000e+02,3.000000e-01,4.010956e+05,2.500000e+00,1.250000e+00


### Q.1. How many columns?

In [12]:
len(jan_df.columns)

19

### Q.2. Standard Deviation of the trips duration in Jan 2022?

In [13]:
jan_df.sample()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1823343,1,2022-01-24 18:35:59,2022-01-24 18:37:59,2.0,0.6,1.0,N,48,68,2,4.0,3.5,0.5,0.0,0.0,0.3,8.3,2.5,0.0


In [14]:
jan_df['duration'] = jan_df.tpep_dropoff_datetime - jan_df.tpep_pickup_datetime

In [16]:
jan_df.duration.describe()

count                      2463931
mean     0 days 00:14:12.732175129
std      0 days 00:46:26.718308266
min              -3 days +14:37:36
25%                0 days 00:06:19
50%                0 days 00:10:11
75%                0 days 00:16:10
max                5 days 21:53:11
Name: duration, dtype: object

### Question 3: Fraction of the records left after dropping the outliers

In [40]:
jan_df['duration'] = jan_df.duration.apply(lambda td : td.total_seconds()/60)
jan_df.duration.describe()

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64

In [46]:
jan_df = jan_df[ (jan_df.duration >= 1) & ( jan_df.duration <=60)]
jan_df.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
count,2.421440e+06,2.354481e+06,2.421440e+06,2.354481e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.354481e+06,2.354481e+06,2.421440e+06
mean,1.700020e+00,1.389813e+00,5.374377e+00,1.359217e+00,1.660323e+02,1.636259e+02,1.189319e+00,1.266161e+01,1.014453e+00,4.940617e-01,2.371678e+00,3.636815e-01,2.974866e-01,1.888460e+01,2.301668e+00,8.057827e-02,1.267128e+01
std,4.705126e-01,9.836160e-01,5.524458e+02,5.585660e+00,6.535271e+01,7.070981e+01,4.876703e-01,2.579900e+02,1.234236e+00,7.060493e-02,2.697403e+00,1.637246e+00,3.859704e-02,2.581265e+02,7.097402e-01,3.084765e-01,8.999282e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-3.350000e+02,-4.500000e+00,-5.000000e-01,-1.252200e+02,-3.140000e+01,-3.000000e-01,-2.533000e+02,-2.500000e+00,-1.250000e+00,1.000000e+00
25%,1.000000e+00,1.000000e+00,1.060000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,3.000000e-01,1.130000e+01,2.500000e+00,0.000000e+00,6.433333e+00
50%,2.000000e+00,1.000000e+00,1.760000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,2.000000e+00,0.000000e+00,3.000000e-01,1.463000e+01,2.500000e+00,0.000000e+00,1.023333e+01
75%,2.000000e+00,1.000000e+00,3.120000e+00,1.000000e+00,2.340000e+02,2.360000e+02,1.000000e+00,1.350000e+01,2.500000e+00,5.000000e-01,3.000000e+00,0.000000e+00,3.000000e-01,1.980000e+01,2.500000e+00,0.000000e+00,1.613333e+01
max,6.000000e+00,9.000000e+00,3.061593e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,4.010923e+05,3.350000e+01,3.300000e+00,8.888800e+02,1.933000e+02,3.000000e-01,4.010956e+05,2.500000e+00,1.250000e+00,6.000000e+01


In [47]:
2421440 /2463931 

0.9827547930522406

### Q.4. Dimensionality after OHE

In [51]:
jan_df.sample()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
1099185,1,2022-01-15 20:51:17,2022-01-15 20:54:14,0.0,0.7,1.0,Y,237,263,2,4.5,3.0,0.5,0.0,0.0,0.3,8.3,2.5,0.0,2.95


In [57]:
jan_df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1, inplace=True)

In [58]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [68]:
jan_df[categorical] = jan_df[categorical].astype(str)
train_dicts = jan_df[categorical].to_dict(orient='records')

In [69]:
dict_vectorizer = DictVectorizer()
dict_vectorizer.fit(train_dicts)

DictVectorizer()

In [71]:
matrix_ = dict_vectorizer.transform(train_dicts)
matrix_.shape

(2421440, 515)

### Q.5. RMSE on train

In [74]:
Y = jan_df.duration

In [73]:
LR = LinearRegression()

In [75]:
LR.fit(matrix_, Y)

LinearRegression()

In [80]:
mean_squared_error(Y, LR.predict(matrix_), squared=False)

6.986190135965426

### Q.6 RMSE on validation

In [103]:
feb_df['duration'] = feb_df.tpep_dropoff_datetime - feb_df.tpep_pickup_datetime
feb_df['duration'] = feb_df.duration.apply(lambda td : td.total_seconds() / 60)
feb_df.duration.describe()

count    2.979431e+06
mean     1.565368e+01
std      4.726394e+01
min     -9.833333e-01
25%      6.950000e+00
50%      1.125000e+01
75%      1.783333e+01
max      5.489383e+03
Name: duration, dtype: float64

In [108]:
feb_df = feb_df[(feb_df.duration >=1) & (feb_df.duration <= 60)]

In [110]:
valid_dicts = feb_df[categorical].astype(str).to_dict(orient='records')

In [111]:
matrix_valid = dict_vectorizer.transform(valid_dicts)
matrix_valid.shape

(2918187, 515)

In [112]:
Y_valid = feb_df.duration.values

(2918187,)

In [113]:
mean_squared_error(Y_valid, LR.predict(matrix_valid), squared=False)

7.7863894674836205